In [ ]:
import numpy as np
import pandas as pd 
from scipy import stats
import warnings

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_validate

import matplotlib.pyplot as plt
import seaborn as sns

import time
start_time = time.time()
%matplotlib inline

In [ ]:
# Import the housing dataset as a Pandas Dataframe
df = pd.read_csv('data/kc_house_data.csv')
df.head()

In [ ]:
# Create a summmary of each column in the df
print(df.info())

print('''
yr_renovated and waterfront are only columns containing null values
''')

### Find average house price and base RMSE using average price per sqft

In [ ]:
# Calculates the average price of houses
avg_price_house = df.price.mean()
avg_price_house

In [ ]:
# Calculates the average price per square feet
avg_price_sqft = (df.price / df.sqft_living).mean()
avg_price_sqft

In [ ]:
# EDA checking if square feet is a strong predictor of housing price
price_pred_base = df.sqft_living * avg_price_sqft
rmse_base = mean_squared_error(df.price, price_pred_base, squared=False)

# Very high rmse, livable square footage isn't a strong predictor
round(rmse_base, 0)

## Data Cleaning

In [ ]:
# Set Nan values of 'waterfront' and 'year_renovated' columns to 0
df.loc[df.waterfront.isna()==True, 'waterfront'] = 0
df.loc[df.yr_renovated.isna()==True, 'yr_renovated'] = 0

#Set all sqft_basement values of '?' to 0, then convert to floats.
df.loc[df.sqft_basement=='?', 'sqft_basement'] = 0
df.sqft_basement = df.sqft_basement.astype(float)

# Convert 'date' to a datetime object and use these to create a 'year' column
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].apply(lambda date: date.year)

# Create an 'age' column to specify how old a house was at sale
df['age'] = df['year'] - df['yr_built']

#Drop unnecessary 'id', 'yr_built', 'year', and 'date' columns
cols_to_drop = ['id', 'yr_built', 'year', 'date']
df.drop(cols_to_drop, axis=1, inplace=True)

#Drop rows that contain null values in the 'view' column
df.dropna(subset = ['view'], inplace = True)

## Handling Outliers

In [ ]:
# assigning variable for columns we would like to check for outliers
target_cols = ['bathrooms', 'bedrooms', 'sqft_living']

# assigning variable to predefined outlier column names
outlier_cols = ['outlier_' + col for col in target_cols]
z_scores = [stats.zscore(df[col]) 
                for col in target_cols]

# for each column, checking to see if the z-score is above or below 3 standard deviations
for outlier_col, z_score in zip(outlier_cols, z_scores):
    df[outlier_col] = ((z_score > 3) | (z_score <-3))

# initializes an empty query string 
query_empty = '({} == False)&'*(len(target_cols))

# fills in the outlier column names
query = query_empty[:-1].format(*outlier_cols)

# selecting rows that do not have any outliers
df = df.query(query)

# dropping the outlier columns that we just made
df = df.drop(columns=outlier_cols)

# Drop the outlier house that contains 33 bedrooms
df = df[df['bedrooms'] != 33]

# Drop houses that were bought before construction
df = df[df['age'] != -1]

In [ ]:
# plotting a histogram to visualize the # of homes sold by # of bedrooms
df.hist('bedrooms');
print('There is a relatively normal distribution of bedroom sizes')

In [ ]:
# plotting a histogram to visualize the # of homes sold by price
print("Price has a positive skew")
df.hist('price', bins=20);

In [ ]:
# Create a matrix of correlations for each feature set in the dataframe
corr_matrix = df.corr()
# Create a boolean mask for all values on or above the matrix diagonal 
corr_matrix_mask = np.triu(np.ones_like(df.corr(), dtype=bool));

In [ ]:
# Creates a heat map of correlation coefficient
fig, ax = plt.subplots(figsize  =(15,15))
sns.heatmap(
            corr_matrix, 
            ax=ax, 
            annot=True, 
            mask= corr_matrix_mask,
            cbar_kws={"label": "Correlation", "orientation": "horizontal", "pad": .2, "extend": "both"}
);

In [ ]:
# Price will be our target value to predict, so we'll zero in on its correlations.
plt.figure(figsize=(4,10))
heatmap = sns.heatmap(df.corr()[['price']].sort_values(by='price', ascending=False), vmin=-1, vmax=1, linewidths=1, linecolor='black', annot=True, fmt='.2g', cmap="gist_heat")

In [ ]:
# Creating visualizations for each of our features as they relate to the price of houses sold in our dataset

x = df.drop('price', axis=1)
y= df.price
fig, axes = plt.subplots(ncols=3, nrows=6, figsize=(12, 12))
fig.set_tight_layout(True)
for index, col in enumerate(x.columns): 
    ax = axes[index//3][index%3]
    ax.scatter(x[col], y, alpha=0.2)
    ax.set_xlabel(col)
    ax.set_ylabel("House price")

### Average Sqft Per Price Range

In [ ]:
# Creates the binning intervals for price
bins = pd.interval_range(0,df.price.max(), freq=150000)

# Creates a column where price values are sorted into bins
df['price_bins'] = pd.cut(df.price, bins)

# Finds the mean square footage for each price bin for each price bin
grouped_means = df.groupby("price_bins")['sqft_living'].mean()
grouped_means = grouped_means.reset_index().dropna(subset=['sqft_living'])

# Breaks up the bins into a readable format for plotting
x_labels = [str(interval.left/1000000) + ' - ' + str(interval.right/1000000)
            for interval in grouped_means['price_bins']
           ]
# Overwrite raw bins with readable format for easy plotting
grouped_means['price_bins'] = x_labels

# Initializes plot and style parameters
plt.rc('font', size=12) 
sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize = (15, 5))

# Plots price bins and average livable space
ax = sns.barplot(y='sqft_living',x='price_bins', data=grouped_means)

# Modify plot label attributes
plt.xticks(rotation=90)
ax.set(xlabel='Price Range ($ Millions)', ylabel='Avg. Livable Space (Sqft)')
plt.title("Average Livable Space by Price Range")

### Average price by Grade

In [ ]:
# Creating a visualization to understand the mean price of homes sold by each value for grade

# Initializes plot and style parameters
sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize = (15, 5))

# Creates a bar plot of our target params
x = df['grade']
y = df['price']
ax = sns.barplot(x=x,y=y, data=df)

# Modify plot label attributes
ax.ticklabel_format(style='plain', axis='y')
ax.set(xlabel='Grade', ylabel='Avg. Price')
plt.title("Average Home Price by Grade")

### Average Price by bathrooms

In [ ]:
# Creating a visualization to understand the mean price of homes sold by number of bathrooms

# Initializes plot and style parameters
sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize = (15, 5))

# Creates a bar plot of our target params
x = df['bathrooms']
y = df['price']
ax = sns.barplot(x=x,y=y, data=df)

# Modify plot label attributes
ax.ticklabel_format(style='plain', axis='y')
ax.set(xlabel='Bathroom', ylabel='Avg. Price')
plt.title("Average Home Price by Number of Bathrooms")
plt.show()

### Average Price by View

In [ ]:
# Creating a visualization to understand the mean price of homes sold by number of views

# Initializes plot and style parameters
sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize = (15, 5))

# Creates a bar plot of our target params
x = df['view']
y = df['price']
ax = sns.barplot(x=x,y=y, data=df)

# Modify plot label attributes
ax.ticklabel_format(style='plain', axis='y')
ax.set(xlabel='View', ylabel='Avg. Price')
plt.title("Average Home Price by View")

### Average Price by Condition

In [ ]:
# Creating a visualization to understand the mean price of homes sold by condition

# Initializes plot and style parameters
sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize = (15, 5))

# Creates a bar plot of our target params
x = df['condition']
y = df['price']
ax = sns.barplot(x=x,y=y, data=df)

# Modify plot label attributes
ax.ticklabel_format(style='plain', axis='y')
ax.set(xlabel='condition', ylabel='Avg. Price')
plt.title("Average Home Price by Condition")

### Average Price by Waterfront

In [ ]:
# Creating a visualization to understand the mean price of homes sold based on whether or not they are on the water

# Initializes plot and style parameters
sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize = (15, 5))

# Creates a bar plot of our target params
x = df['waterfront']
y = df['price']
ax = sns.barplot(x=x,y=y, data=df)

# Modify plot label attributes
ax.ticklabel_format(style='plain', axis='y')
ax.set_xticklabels(['Not On Waterfront', 'On Waterfront'])
ax.set(xlabel='waterfront', ylabel='Avg. Price')
plt.title("Average Home Price by Waterfront")
plt.show()

### Scatterplot comparing sqft of neighbors to price

In [ ]:
# Creating a visualization to understand the mean price of homes sold based on the 15 closest neighbors

# Initializes plot and style parameters
fig, ax = plt.subplots(figsize = (15, 5))

# Creates a bar plot of our target params
x = df['sqft_living15']
y = df['price']

ax = sns.regplot(x=x,y=y, data=df, line_kws={'color': 'r'})

# Modify plot label attributes
ax.ticklabel_format(style='plain', axis='y')
ax.set(xlabel='Sqft of 15 closest Neighbors ', ylabel='Avg. Price')
plt.title("Home prices based on sqft of 15 closest Neighbors")

plt.show()

### Barplot comparing sqft of neighbors to price

In [ ]:
# Creates the binning intervals for price
bins = pd.interval_range(0,df.price.max(), freq=150000)

# Creates a column where price values are sorted into bins
df['price_bins'] = pd.cut(df.price, bins)

# Finds the mean square footage for the 15 closest neighbors for each price bin for each price bin
grouped_means = df.groupby("price_bins")['sqft_living15'].mean()
grouped_means = grouped_means.reset_index().dropna(subset=['sqft_living15'])

# Breaks up the bins into a readable format for plotting
x_labels = [str(interval.left/1000000) + ' - ' + str(interval.right/1000000)
            for interval in grouped_means['price_bins']
           ]

# Overwrite raw bins with readable format for easy plotting
grouped_means['price_bins'] = x_labels

# Initializes plot and style parameters
plt.rc('font', size=12) 
sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize = (15, 5))

# Plots price bins and average livable space
ax = sns.barplot(y='sqft_living15',x='price_bins', data=grouped_means)

# Modify plot label attributes
plt.xticks(rotation=90)
ax.set(xlabel='Price Range ($ Millions)', ylabel='Avg. Sqft of 15 closet neighbors')
plt.title("Average Sqft of 15 closest neighbors");

# Model Building

## Baseline Model

In [ ]:
df = df.drop('price_bins', axis = 1)

In [ ]:
# Drop the new 'age' column and our target column 'price' for the independent features
X = df.drop(['price', 'age'], axis = 1)

# Set our dependent variable as price
y = df.price
  
# Split up our independent and dependent variables into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Trains our model on our baseline values
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_train, y_train)

# predicting test prices based on the X_test
y_predict_dummy_mean = lm_dummy_mean.predict(X_test)

# Find the error of our dummy model
rmse_lr0 = mean_squared_error(y_test, y_predict_dummy_mean, squared=False)
print(r2_score(y_test, y_predict_dummy_mean), rmse_lr0)

### Model 1:  including 'age'

In [ ]:
# plotting the # of homes sold by their age
df.hist('age');

In [ ]:
df.info()

In [ ]:
# Drop the dependent variable from the independent columns
X = df.drop(['price'], axis = 1)

# Set our dependent variable as price
y = df['price']

# Split up our independent and dependent variables into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [ ]:
# Initialize an empty regression model
lr = LinearRegression()

# Fits the model to our training dataset
lr.fit(X_train, y_train)

# Predict price with the trained model
pred_lr1 = lr.predict(X_test)

# Get the coefficient of determination for training and test data
train_score_lr1 = lr.score(X_train, y_train)
test_score_lr1 = lr.score(X_test, y_test)

In [ ]:
# Take a peak at model coef
lr.coef_[0]

In [ ]:
# Baseline housing cost without features
lr.intercept_

In [ ]:
# Find the error of our predicted y test values
rmse_lr1 = mean_squared_error(y_test, pred_lr1, squared=False)

In [ ]:
train_score_lr1, test_score_lr1, rmse_lr1

In [ ]:
# Cross validate our model and find the mean of the training scores
scores_simple_1 = cross_validate(
                    lr, X_train, y_train, cv=5, 
                    return_train_score=True
)

In [ ]:
simple_1_mean = np.mean(scores_simple_1['train_score'])
simple_1_mean_test = np.mean(scores_simple_1['test_score'])

In [ ]:
# Checking the QQ Plot to understand the distribution of residuals
residuals1 = (y_test - pred_lr1)
sm.graphics.qqplot(residuals1, dist=stats.norm, line="45", fit=True);

In [ ]:
# Checking for homoscedasticity 

residuals1 = (y_test - pred_lr1)
fig, ax = plt.subplots()
ax.scatter(pred_lr1, residuals1, alpha=.1)
ax.plot(pred_lr1, [0 for i in range(len(X_test))])
ax.set_xlabel('Predicted Value')
ax.set_ylabel('Actual - Predicted Value');

In [ ]:
# Checking to see how our regression line fits our data
sns.regplot(x = y_test, y = pred_lr1, data = df, line_kws={'color':'r'}, scatter_kws={'alpha':0.1});

### Model 2: log transform 'price'

In [ ]:
# Drop the dependent variable from the independent columns
X = df.drop(['price'], axis = 1)

# Set our dependent variable as the natural log of price
y = np.log(df['price'])

# Split up our independent and dependent variables into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize an empty regression model
lr = LinearRegression()

# Fits the model to our training dataset
lr.fit(X_train, y_train)

# Predict price with the trained model
pred_lr2 = lr.predict(X_test)

In [ ]:
# Using the score method to see how well our model performed based on how we trained it
train_score_lr2 = lr.score(X_train, y_train)
test_score_lr2 = lr.score(X_test, y_test)

In [ ]:
# Take a peak at model coef
lr.coef_[0] 

In [ ]:
# Baseline housing cost without features
lr.intercept_

In [ ]:
# Normalizing our price to get an RMSE
rmse_lr2 = mean_squared_error(np.exp(y_test), np.exp(pred_lr2), squared=False)

In [ ]:
train_score_lr2, test_score_lr2, rmse_lr2 

In [ ]:
# Checking the QQ Plot to understand the distribution of residuals
residuals2 = (y_test - pred_lr2)
sm.graphics.qqplot(residuals2, dist=stats.norm, line="45", fit=True);

In [ ]:
# Checking for homoscedasticity 
residuals2 = (y_test - pred_lr2)
fig, ax = plt.subplots()
ax.scatter(pred_lr2, residuals2, alpha=.1)
ax.plot(pred_lr2, [0 for i in range(len(X_test))])
ax.set_xlabel('Predicted Value')
ax.set_ylabel('Actual - Predicted Value');

In [ ]:
# Checking to see how our regression line fits our data
sns.regplot(x = y_test, y = pred_lr2, data = df, line_kws={'color':'r'});

In [ ]:
# Cross validate our model and find the mean of the training scores
scores_simple_2 = cross_validate(
                    lr, X_train, y_train, cv=5, 
                    return_train_score=True
)

In [ ]:
simple_2_mean = np.mean(scores_simple_2['train_score'])
simple_2_mean_test = np.mean(scores_simple_2['test_score'])

### Checking the VIF Score

In [ ]:
# Making a constant column
df_temp = sm.add_constant(df)

# Checking the Multicollinearity between the features
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_temp.values, i) for i in range(df_temp.values.shape[1])]
vif["features"] = df_temp.columns

print(vif.round(1))

### Model 3:  creating dummy columns for zip code

In [ ]:
# Adds dummy zipcode columns
df = df.join(pd.get_dummies(df['zipcode'], prefix = 'x', drop_first = True))
df.drop('zipcode', axis=1, inplace=True)

In [ ]:
# Drop the dependent variable from the independent columns
X = df.drop(['price'], axis = 1)

# Set our dependent variable as the natural log of price
y = np.log(df['price'])

# Split up our independent and dependent variables into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize an empty regression model
lr = LinearRegression()

# Fits the model to our training dataset
lr.fit(X_train, y_train)

In [ ]:
# Predict price with the trained model
pred_lr3 = lr.predict(X_test)

In [ ]:
# Using the score method to see how well our model performed based on how we trained it
train_score_lr3 = lr.score(X_train, y_train)
test_score_lr3 = lr.score(X_test, y_test)

In [ ]:
# Take a peak at model coef
lr.coef_[0]

In [ ]:
# Baseline housing cost without features
lr.intercept_

In [ ]:
# Normalizing our price to get an RMSE
rmse_lr3 = mean_squared_error(np.exp(y_test), np.exp(pred_lr3), squared=False)

In [ ]:
train_score_lr3, test_score_lr3, rmse_lr3

In [ ]:
# Making an OLS table to check for feature significance
X = sm.add_constant(X)
sm.OLS(y, X).fit().summary()

In [ ]:
# Cross validate our model and find the mean of the training scores
scores_simple_3 = cross_validate(
                    lr, X_train, y_train, cv=5, 
                    return_train_score=True
)

In [ ]:
simple_3_mean = np.mean(scores_simple_3['train_score'])
simple_3_mean_test = np.mean(scores_simple_3['test_score'])

In [ ]:
# Making the QQ Plot
residuals3 = (y_test - pred_lr3)
sm.graphics.qqplot(residuals3, dist=stats.norm, line="45", fit=True);

In [ ]:
# Checking for homoscedasticity 
residuals3 = (y_test - pred_lr3)
fig, ax = plt.subplots()
ax.scatter(pred_lr3, residuals3, alpha=.1)
ax.plot(pred_lr3, [0 for i in range(len(X_test))])
ax.set_xlabel('Predicted Value')
ax.set_ylabel('Actual - Predicted Value');

In [ ]:
# Checking to see how our regression line fits our data
sns.regplot(x = y_test, y = pred_lr3, data = df, line_kws={'color':'r'});

### Model 4:  dropping features with high p_values


In [ ]:
# Use summary above to drop features with high p_values
X = df.drop(['price', 'x_98002', 'x_98003', 'sqft_basement', 'bedrooms'], axis = 1)

# Set our dependent variable as the natural log of price
y = np.log(df['price'])

# Split up our independent and dependent variables into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize an empty regression model
lr = LinearRegression()

# Fits the model to our training dataset
lr.fit(X_train, y_train)

In [ ]:
# Predict price with the trained model
pred_lr4 = lr.predict(X_test)

In [ ]:
# Using the score method to see how well our model performed based on how we trained it
train_score_lr4 = lr.score(X_train, y_train)
test_score_lr4 = lr.score(X_test, y_test)

In [ ]:
# Take a peak at model coef
lr.coef_[0]

In [ ]:
# Baseline housing cost without features
lr.intercept_

In [ ]:
# Normalizing our price to get an RMSE
rmse_lr4 = mean_squared_error(np.exp(y_test), np.exp(pred_lr4), squared=False)

In [ ]:
train_score_lr4, test_score_lr4, rmse_lr4 

In [ ]:
# Checking the OLS for values of feature significance
X = sm.add_constant(X)
sm.OLS(y, X).fit().summary()

In [ ]:
# Cross validate our model and find the mean of the training scores
scores_simple_4 = cross_validate(
                    lr, X_train, y_train, cv=5, 
                    return_train_score=True
)

In [ ]:
simple_4_mean = np.mean(scores_simple_4['train_score'])
simple_4_mean_test = np.mean(scores_simple_4['test_score'])

In [ ]:
# Checking the normal distribution of our residuals
residuals4 = (y_test - pred_lr4)
sm.graphics.qqplot(residuals4, dist=stats.norm, line="45", fit=True);

In [ ]:
# Checking for homoscedasticity 
residuals4 = (y_test - pred_lr4)
fig, ax = plt.subplots()
ax.scatter(pred_lr4, residuals4, alpha=.1)
ax.plot(pred_lr4, [0 for i in range(len(X_test))])
ax.set_xlabel('Predicted Value')
ax.set_ylabel('Actual - Predicted Value');

In [ ]:
# Checking to see how our regression line fits our data
sns.regplot(x = y_test, y = pred_lr4, data = df, line_kws={'color':'r'});

### Model 5: adding back the zipcodes we dropped in our previous model

In [ ]:
# Drop dependent variable and non-relevant features for independent variable
X = df.drop(['price', 'sqft_basement', 'bedrooms'], axis = 1)

# Set independent variable to the natural log of price
y = df['price'] 
y = np.log(y)

# Split our training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize an empty regression model
lr = LinearRegression()

# Fits the model to our training dataset
lr.fit(X_train, y_train)

In [ ]:
# Predict price with the trained model
pred_lr5 = lr.predict(X_test)

In [ ]:
# Using the score method to see how well our model performed based on how we trained it
train_score_lr5 = lr.score(X_train, y_train)
test_score_lr5 = lr.score(X_test, y_test)

In [ ]:
# Take a peak at model coef
lr.coef_[0]

In [ ]:
# Baseline housing cost without features
lr.intercept_

In [ ]:
# Normalizing our price to get an RMSE
rmse_lr5 = mean_squared_error(np.exp(y_test), np.exp(pred_lr5), squared=False)

In [ ]:
train_score_lr5, test_score_lr5, rmse_lr5

In [ ]:
# Checking the OLS for values of feature significance
X = sm.add_constant(X)
sm.OLS(y, X).fit().summary()

In [ ]:
# Cross validate our model and find the mean of the training scores
scores_simple_5 = cross_validate(
                    lr, X_train, y_train, cv=5, 
                    return_train_score=True
)

In [ ]:
simple_5_mean = np.mean(scores_simple_5['train_score'])
simple_5_mean_test = np.mean(scores_simple_5['test_score'])

In [ ]:
# Checking the normal distribution of our residuals
residuals5 = (y_test - pred_lr5)
sm.graphics.qqplot(residuals5, dist=stats.norm, line="45", fit=True);

In [ ]:
# Checking for homoscedasticity 
residuals5 = (y_test - pred_lr4)
fig, ax = plt.subplots()
ax.scatter(pred_lr5, residuals5, alpha=.1)
ax.plot(pred_lr5, [0 for i in range(len(X_test))])
ax.set_xlabel('Predicted Value')
ax.set_ylabel('Actual - Predicted Value');

In [ ]:
# Checking to see how our regression line fits our data
sns.regplot(x = y_test, y = pred_lr5, data = df, line_kws={'color':'r'});

### Model 6: Our best LR model

In [ ]:
# Drop dependent variable and non-relevant features for independent variable
X = df.drop(['price', 'sqft_basement', 'bedrooms', 'sqft_above'], axis = 1)
y = df['price'] 
y = np.log(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize an empty regression model
lr = LinearRegression()

# Fits the model to our training dataset
lr.fit(X_train, y_train)

# Predict price with the trained model
pred_lr6 = lr.predict(X_test)

# Get the coefficient of determination for training and test data
train_score_lr6 = lr.score(X_train, y_train)
test_score_lr6 = lr.score(X_test, y_test)

# Take a peak at model coef
lr.coef_[0]

# Baseline housing cost without features
lr.intercept_

# Normalizing our price to get an RMSE
rmse_lr6 = mean_squared_error(np.exp(y_test), np.exp(pred_lr6), squared=False)

In [ ]:
train_score_lr6, test_score_lr6, rmse_lr6

In [ ]:
# Making an OLS table to check for feature significance
X = sm.add_constant(X)
sm.OLS(y, X).fit().summary()

In [ ]:
# Cross validate our model and find the mean of the training scores
scores_simple_6 = cross_validate(
                    lr, X_train, y_train, cv=5, 
                    return_train_score=True
)

In [ ]:
simple_6_mean = np.mean(scores_simple_6['train_score'])
simple_6_mean_test = np.mean(scores_simple_6['test_score'])

In [ ]:
# Making the QQ Plot
residuals6 = (y_test - pred_lr6)
sm.graphics.qqplot(residuals6, dist=stats.norm, line="45", fit=True);

In [ ]:
# Checking for homoscedasticity 
residuals6 = (y_test - pred_lr6)
fig, ax = plt.subplots()
ax.scatter(pred_lr6, residuals6, alpha=.1)
ax.plot(pred_lr6, [0 for i in range(len(X_test))])
ax.set_xlabel('Predicted Value')
ax.set_ylabel('Actual - Predicted Value');

In [ ]:
# Checking to see how our regression line fits our data
sns.regplot(x = y_test, y = pred_lr6, data = df, line_kws={'color':'r'});

### Model 7:  Polynomial Regression

In [ ]:
# Drop dependent variable and non-relevant features for independent variable
X = df.drop(['price', 'sqft_basement', 'sqft_lot15', 'sqft_above', 'bathrooms', 'sqft_living', 'grade'], axis = 1)
y = df['price'] 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
poly = PolynomialFeatures(2)

In [ ]:
# Pass in our training dataset
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

In [ ]:
# Initialize an empty regression model
lr = LinearRegression()

In [ ]:
# Fits the model to our training dataset
lr.fit(X_train_poly, y_train)

In [ ]:
# Get the coefficient of determination for training and test data
score_train_poly = lr.score(X_train_poly, y_train)
score_test_poly = lr.score(X_test_poly,y_test)

In [ ]:
# Predict price with the trained model
pred_poly = lr.predict(X_test_poly)

In [ ]:
# Normalizing our price to get an RMSE
rmse_poly_7 = mean_squared_error(y_test, pred_poly, squared=False)

In [ ]:
score_train_poly, score_test_poly, rmse_poly_7

In [ ]:
# Cross validate our model and find the mean of the training scores
scores_complex_1 = cross_validate(
                    lr, X_train_poly, y_train, cv=5, 
                    return_train_score=True
)

In [ ]:
scores_complex_1_train = np.mean(scores_complex_1['train_score'])
scores_complex_1_test = np.mean(scores_complex_1['test_score'])

In [ ]:
# One of the test scores is negative, will not use model
scores_complex_1

In [ ]:
# Checking the normal distribution of our residuals
residuals = (y_test - pred_poly)
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True);

In [ ]:
# Checking for homoscedasticity 
residuals = (y_test - pred_poly)
fig, ax = plt.subplots()
ax.scatter(pred_poly, residuals, alpha=.1)
ax.plot(pred_poly, [0 for i in range(len(X_test))])
ax.set_xlabel('Predicted Value')
ax.set_ylabel('Actual - Predicted Value');

In [ ]:
# Checking to see how our regression line fits our data
sns.regplot(x = y_test, y = pred_poly, data = df, line_kws={'color':'r'});

### Model 8: Linear Regression with Feature Engineering

In [ ]:
# Feature Engineering Making 4 new colmns - here we are turning grade into a polynomial and multiplying it by existing features which improves the correlation to price compared to using the initial features.
df['grade_sqft_living'] = (df.grade**2) * df.sqft_living
df['grade_sqft_above'] = (df.grade**2) * df.sqft_above
df['grade_sqft_living15'] = (df.grade**2) * df.sqft_living15
df['grade_bathrooms'] = (df.grade**2) * df.bathrooms

In [ ]:
# Droping columns that are not going to be used 
X = df.drop(['price', 'sqft_basement', 'sqft_lot15', 'sqft_above', 'bathrooms', 'sqft_living', 'grade'], axis = 1)
y = df['price'] 
y = np.log(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize an empty regression model
lr = LinearRegression()

# Fits the model to our training dataset
lr.fit(X_train, y_train)

In [ ]:
# Predict price with the trained model
pred_lr8 = lr.predict(X_test)

In [ ]:
# Get the coefficient of determination for training and test data
train_score_lr8 = lr.score(X_train, y_train)
test_score_lr8 = lr.score(X_test, y_test)

In [ ]:
# Take a peak at model coef
lr.coef_[0]

In [ ]:
# Baseline housing cost without features
lr.intercept_

In [ ]:
# Normalizing our price to get an RMSE
rmse_lr8 = mean_squared_error(np.exp(y_test), np.exp(pred_lr8), squared=False)

In [ ]:
train_score_lr8, test_score_lr8, rmse_lr8 

In [ ]:
# Making an OLS table to check for feature significance
X = sm.add_constant(X)
sm.OLS(y, X).fit().summary()

In [ ]:
# Cross validate our model and find the mean of the training scores
simple_7_mean = cross_validate(
                    lr, X_train, y_train, cv=5, 
                    return_train_score=True
)

In [ ]:
scores_simple_7 = np.mean(simple_7_mean['train_score'])
simple_7_mean_test = np.mean(simple_7_mean['test_score'])

In [ ]:
# Checking the normal distribution of our residuals
residuals8 = (y_test - pred_lr8)
sm.graphics.qqplot(residuals8, dist=stats.norm, line="45", fit=True);

In [ ]:
# Checking for homoscedasticity 
residuals8 = (y_test - pred_lr8)
fig, ax = plt.subplots()
ax.scatter(pred_lr8, residuals8, alpha=.1)
ax.plot(pred_lr8, [0 for i in range(len(X_test))])
ax.set_xlabel('Predicted Value')
ax.set_ylabel('Actual - Predicted Value');

In [ ]:
# Checking to see how our regression line fits our data
sns.regplot(x = y_test, y = pred_lr8, data = df, line_kws={'color':'r'});

### Model 9: A Polynomial model with feature engineering

In [ ]:
df['grade_sqft_living'] = (df.grade**2) * df.sqft_living
df['grade_sqft_above'] = (df.grade**2) * df.sqft_above
df['grade_sqft_living15'] = (df.grade**2) * df.sqft_living15
df['grade_bathrooms'] = (df.grade**2) * df.bathrooms

In [ ]:
# Drop dependent column and those that were used in feature engineering
X = df.drop(['price', 'sqft_basement', 'sqft_lot15', 'sqft_above', 'bathrooms', 'sqft_living', 'grade'], axis = 1)
y = df['price'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize empty polynomial object- 2 degrees
poly = PolynomialFeatures(2)

In [ ]:
# Pass in our training dataset
X_train_poly = poly.fit_transform(X_train)

In [ ]:
# Pass in our test dataset
X_test_poly = poly.transform(X_test)

In [ ]:
# Initialize an empty regression model
lr = LinearRegression()

In [ ]:
# Use poly training X and Y data for linear regression model
lr.fit(X_train_poly, y_train)

In [ ]:
# Get the coefficient of determination for training and test data
score_train_poly = lr.score(X_train_poly, y_train)
score_test_poly = lr.score(X_test_poly,y_test)

In [ ]:
# Predict price with the trained model
pred_poly = lr.predict(X_test_poly)

In [ ]:
# Calculating the error of our predicted price versus actual
rmse_poly_9 = mean_squared_error(y_test, pred_poly, squared=False)

In [ ]:
score_train_poly, score_test_poly, rmse_poly_9 

In [ ]:
# Pass in our training dataset
X_poly = poly.fit_transform(X_train)

# Initialize an empty regression model
model_1 = LinearRegression()

# Cross validate our model and find the mean of the training scores
scores_complex_2 = cross_validate(
                    model_1, X_poly, y_train, cv=5, 
                    return_train_score=True
)

In [ ]:
scores_complex_2_train = np.mean(scores_complex_2['train_score'])
scores_complex_2_test = np.mean(scores_complex_2['test_score'])

In [ ]:
# Checking the normal distribution of our residuals
residuals = (y_test - pred_poly)
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True);

In [ ]:
# Checking for homoscedasticity 
residuals = (y_test - pred_poly)
fig, ax = plt.subplots()
ax.scatter(pred_poly, residuals, alpha=.1)
ax.plot(pred_poly, [0 for i in range(len(X_test))])
ax.set_xlabel('Predicted Value')
ax.set_ylabel('Actual - Predicted Value');

In [ ]:
# Checking to see how our regression line fits our data
sns.regplot(x = y_test, y = pred_poly, data = df, line_kws={'color':'r'});

In [ ]:
# plotting the RMSE's for each of the models we created - the RMSE's represent the difference between our actual test scores vs. our predicted test scores

models = ['Using Price/Sqft', 'Model_Base', 'Model_1', 'Model_2', 'Model_3', 'Model_4', 'Model_5', 
          'Model_6', 'Model_7', 'Model_8', 'Model_9']
RMSE = [rmse_base, rmse_lr0, rmse_lr1, rmse_lr2, rmse_lr3, rmse_lr4, 
        rmse_lr5, rmse_lr6, rmse_poly_7, rmse_lr8, rmse_poly_9]
plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(figsize=(15,12))
ax.plot(models, RMSE);
ax.scatter(models, RMSE);
ax.set_ylabel('Root Mean Squared Error, Dollars')
ax.set_title('Average Dollar Error by Model');

In [ ]:
# Plotting Error of base price / sqft and Model 9
fig, ax = plt.subplots(figsize=(8,8))
x = ['Base Price/Sqft', 'Our Model']
y = [rmse_base, rmse_poly_9]
ax.bar(x, y)
ax.set_ylabel('Average Error (Dollars)')
ax.set_title('Error Using Price/Sft vs Our Model');

In [ ]:
# Making an array of all the train score means for our cross e-vals
x_ = np.array([simple_1_mean,
    simple_2_mean,
    simple_3_mean,
    simple_4_mean,
    simple_5_mean,
    simple_6_mean,
    scores_complex_1_train,
    scores_simple_7,
    scores_complex_2_train])

In [ ]:
# counter
p = 0

# Using a for loop to print out every train score
for i in x_:
    p += 1
    print(f'The Train Score is {round(i, 3)} in model {p}')
print('\n')
# Getting the train score that was the highest
print(f'The Max Train score is {x_.max()}')

In [ ]:
# Making an array of all the test scores for our cross e-vals
y_ = np.array([simple_1_mean_test,
    simple_2_mean_test,
    simple_3_mean_test,
    simple_4_mean_test,
    simple_5_mean_test,
    simple_6_mean_test,
    scores_complex_1_test,
    simple_7_mean_test,
    scores_complex_2_test])

In [ ]:
# Counter
m = 0

# Using a for loop to print out every test score
for i in y_:
    m += 1
    # to check if the test score is positive
    if i <= 0:
        print(f'{round(i, 3)} is not accurate for model {m}\n')
    else:
        print(f'The Test score is {round(i, 3)} in model {m}\n')
print('\n')

# Return the hightest test score
print(f'The highest test score is {y_.max()}')    

In [ ]:
# Making an arrray of the RMSE
z_ = np.array([rmse_lr0,
               rmse_lr1,
               rmse_lr2,
               rmse_lr3,
               rmse_lr4,
               rmse_lr5,
               rmse_lr6,
               rmse_poly_7,
               rmse_lr8,
               rmse_poly_9]
             )

In [ ]:
# counter
b = 0

# Using a for loop to return the RMSE for all the models
for i in z_:
    m += 1
    print(f'The RMSE score is {round(i, 3)} in model {m}\n')
print('\n')
# Getting the lowest RMSE
print(f'The lowest RMSE score is {z_.min()}')  

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))